In [13]:
classifierName = "JumpingJacksRest.mlmodel"
dataName = "jumpingJacksRest.txt"

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import *
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [15]:
df = pd.read_csv("./Data/" + dataName, sep=",", header=None)

In [16]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.505208,0.171875,0.505208,0.296875,0.463542,0.296875,0.463542,0.366319,0.588542,0.463542,...,0.671875,0.477431,0.796875,0.463542,0.505208,0.421875,0.657986,0.380208,0.796875,1
1,0.505208,0.171875,0.505208,0.269097,0.477431,0.296875,0.463542,0.366319,0.477431,0.463542,...,0.671875,0.463542,0.796875,0.477431,0.505208,0.505208,0.657986,0.532986,0.796875,1
2,0.505208,0.171875,0.505208,0.296875,0.421875,0.296875,0.380208,0.338542,0.407986,0.463542,...,0.671875,0.407986,0.796875,0.505208,0.505208,0.505208,0.671875,0.532986,0.796875,1
3,0.505208,0.171875,0.505208,0.296875,0.463542,0.296875,0.463542,0.366319,0.463542,0.463542,...,0.671875,0.477431,0.796875,0.477431,0.505208,0.463542,0.644097,0.463542,0.796875,1
4,0.505208,0.171875,0.505208,0.296875,0.421875,0.296875,0.380208,0.338542,0.407986,0.463542,...,0.671875,0.421875,0.796875,0.491319,0.505208,0.505208,0.657986,0.529514,0.796875,1


In [17]:
print('Number of Points:', df.shape[0])

Number of Points: 56


In [18]:
X = df.iloc[:, :-1].to_numpy()
Y = df.iloc[:, 28].to_numpy()

In [19]:
scaler = RobustScaler()
# X = scaler.fit_transform(X)

In [20]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

In [21]:
def calcAccuracy(clf, clf_name):
    scores = []
    bestScore = 0
    bestClassifier = None
    bestConfusionMatrix = None
    
    for train_index, test_index in cv.split(X):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        scores.append(score)
        y_pred = clf.predict(X_test)
        
        if score > bestScore:
            bestScore = score
            bestClassifier = clf
            bestConfusionMatrix = confusion_matrix(y_test, y_pred)

    print(clf_name)
    print("Mean Accuracy: " + str(round(np.mean(scores), 4) * 100) + "%")
    print(bestConfusionMatrix)
    print()

In [22]:
calcAccuracy(KNeighborsClassifier(), "K-Nearest Neighbors")
calcAccuracy(MLPClassifier(hidden_layer_sizes=(100, 50)), "Neural Network")
calcAccuracy(RandomForestClassifier(), "Random Forest")

K-Nearest Neighbors
Mean Accuracy: 76.82%
[[6 1]
 [0 4]]

Neural Network
Mean Accuracy: 92.88%
[[5 0]
 [0 6]]

Random Forest
Mean Accuracy: 96.52%
[[7 0]
 [0 4]]



In [23]:
model = RandomForestClassifier()
model.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [24]:
import coremltools

featureNames = ["topX", "topY",
                "neckX", "neckY",
                "rShoulderX", "rShouderY",
                "rElbowX", "rElbowY",
                "rWristX", "rWristY",
                "lShoulderX", "lShoulderY",
                "lElbowX", "lElbowY",
                "lWristX", "lWristY",
                "rHipX", "rHipY",
                "rKneeX", "rKneeY",
                "rAnkleX", "rAnkleY",
                "lHipX", "lHipY",
                "lKneeX", "lKneeY",
                "lAnkleX", "lAnkleY"]
labelName = "label"

coreml_model = coremltools.converters.sklearn.convert(model, featureNames, labelName)
coreml_model.save(classifierName)